In [1]:
import json
import tensorflow as tf
import json
%matplotlib notebook
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
import os
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
data_path = './event_sequence_data_20200607_hour_08.json'

with open(data_path, 'r') as f:
    raw_data = json.load(f)

In [3]:
raw_data[0]
raw_data_small = raw_data[:500000]
neg_data = []
pos_data = []

for r in raw_data_small:
    if r['fraud_prob'] > 0:
        pos_data.append(r)
    else:
        neg_data.append(r)

In [152]:
times = []
event_types = []
labels = []

vocab = ['na', 'start', 'view', 'click', 'install']
str2idx = {u:i for i, u in enumerate(vocab)}
idx2str= np.array(vocab)

for row in raw_data_small:
    _event_type_list, _time_list = list(zip(*row['sequence']))
    _event_type_list = tuple(str2idx[c] for c in _event_type_list)
    # drop missing times for data points 5096332 and 8959733
    # otherwise cause problems in creating embeddings
    if not any(_time_list): 
        continue
        
    labels.append(row['fraud_prob'])    
    event_types.append(_event_type_list)

    times.append(_time_list)    


## Pre-train Discriminator

In [227]:
padding_len = 20 
padding_type = 'post'
truncating_type = 'pre'

padded_event_types = tf.keras.preprocessing.sequence.pad_sequences(
    event_types, 
    padding=padding_type, 
    truncating = truncating_type,
    maxlen=padding_len,
    value=0,
    dtype=object,
)
padded_event_types = padded_event_types.astype(int)
labels = np.array(labels)
train_data_discriminator = padded_event_types,labels

In [261]:
padded_event_types[1]

array([1, 2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [265]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

discriminator = Sequential()
discriminator.add(Embedding(vocab_size, 16, \
                            input_length=padding_len))
discriminator.add(Conv1D(32, 3, activation='relu'))
discriminator.add(MaxPooling1D(5))
discriminator.add(Conv1D(32, 3, activation='relu'))
discriminator.add(GlobalMaxPooling1D())
discriminator.add(Dense(1))
discriminator.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy',metrics=['acc'])
history = discriminator.fit(padded_event_types, labels,
epochs=2, batch_size=500, validation_split=0.2)


# dataset = tf.data.Dataset.from_tensor_slices(train_data_discriminator)
# dataset = dataset.shuffle(BUFFER_SIZE).batch(500, drop_remainder=True)
# discriminator.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy',metrics=['acc'])
# discriminator.fit(dataset)

Train on 400000 samples, validate on 100000 samples
Epoch 1/2
400000/400000 [==============================] - 6s 15us/sample - loss: 0.1397 - acc: 0.9906 - val_loss: 0.1406 - val_acc: 0.9905
Epoch 2/2
400000/400000 [==============================] - 5s 11us/sample - loss: 0.1397 - acc: 0.9906 - val_loss: 0.1406 - val_acc: 0.9905


## Pre-train Generator model

In [270]:
pos_event_types = []
for row in pos_data:
    _event_type_list, _time_list = list(zip(*row['sequence']))
    _event_type_list = tuple(str2idx[c] for c in _event_type_list)
    # drop missing times for data points 5096332 and 8959733
    # otherwise cause problems in creating embeddings
    if not any(_time_list): 
        continue
    pos_event_types.append(_event_type_list)

    times.append(_time_list)    

pos_padded_event_types = tf.keras.preprocessing.sequence.pad_sequences(
    pos_event_types, 
    padding=padding_type, 
    truncating = truncating_type,
    maxlen=padding_len,
    value=0,
    dtype=object,
)
pos_padded_event_types = pos_padded_event_types.astype(int)

print('pos_padded_event_types:{}'.format(pos_padded_event_types.shape))
# auto-regressive data set
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = tf.data.Dataset.from_tensor_slices(pos_padded_event_types)
dataset = dataset.map(split_input_target)

BATCH_SIZE = 12
BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

for input_example, target_example in  dataset.take(1):
    print('input_example:{}'.format(input_example))
    print('target_example:{}'.format(target_example))
input_example.shape


pos_padded_event_types:(4719, 20)
input_example:[[1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
target_example:[[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

TensorShape([12, 19])

In [28]:
vocab_size = len(vocab)
embedding_dim = 16
rnn_units = 32

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model = build_model(vocab_size = len(vocab),
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)
for input_example, target_example in  dataset.take(1):
    print(input_example.shape)
    print(model(input_example).shape)

# model.compile(optimizer='adam', loss=loss)

# checkpoint_dir = './training_checkpoints'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
# checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_prefix,
#     save_weights_only=True)

# EPOCHS = 2
# history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


(10, 19)
(10, 19, 5)


In [62]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


## Generator model

In [254]:
checkpoint_dir = './training_checkpoints'

tf.train.latest_checkpoint(checkpoint_dir)
generator = build_model(vocab_size, embedding_dim, rnn_units, batch_size=50)

generator.load_weights(tf.train.latest_checkpoint(checkpoint_dir))


## Roll-out policy

In [257]:
def generate_event(model, batch_size=50, length=20):
    states = [1] * batch_size
    states = tf.expand_dims(states, -1)
    all_actions_one_hot = tf.one_hot(states, depth=5)
    all_predicted_actions = None
    
    model.reset_states()
    for i in range(length):
        predictions = model(states)
        if all_predicted_actions is None:
            all_predicted_actions = predictions
        else:
            all_predicted_actions =tf.keras.layers.concatenate([all_predicted_actions, predictions], axis=1)
        
        # sample an action
        predictions = tf.squeeze(predictions)
        predicted_idx = tf.random.categorical(predictions, num_samples=1)
        predicted_idx_one_hot = tf.one_hot(predicted_idx, depth=5)
        all_actions_one_hot = tf.concat([all_actions_one_hot, predicted_idx_one_hot], axis=1)
        
        # pass the predicted action as the next input to the model
        states = predicted_idx

    return  all_actions_one_hot, all_predicted_actions


<img src="train-process.png">


In [268]:

optimizer = tf.keras.optimizers.Adam()
loss_history = []
g_steps = 10
d_steps = 10
# Generator training

for i in range(g_steps):
    with tf.GradientTape() as tape:
        all_actions_one_hot, all_predicted_actions = generate_event(generator, length=20)
        all_actions_one_hot = all_actions_one_hot[:, 1:,]

        # policy gradient loss
        negative_likelihoods = tf.keras.losses.categorical_crossentropy\
            (all_actions_one_hot, all_predicted_actions,from_logits=True)
        in_dis = tf.argmax(all_actions_one_hot, axis = -1)
        q_values = tf.broadcast_to(discriminator(in_dis), negative_likelihoods.shape) # this needs N-time Monte Carlo search
        weighted_negative_likelihoods = tf.multiply(negative_likelihoods, q_values)
        loss = tf.reduce_mean(weighted_negative_likelihoods)

    loss_history.append(loss.numpy())
    print('loss:{}'.format(loss.numpy()))
    grads = tape.gradient(loss, generator.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
# Discriminator training
for i in range(d_steps):
    with tf.GradientTape() as tape:
    all_actions_one_hot, all_predicted_actions = generate_event(generator, length=20)

    



loss:-0.032061103731393814
loss:-0.0318526066839695
loss:-0.03146100416779518
loss:-0.030350029468536377
loss:-0.028879273682832718
loss:-0.028719399124383926
loss:-0.02949499525129795
loss:-0.03095909208059311
loss:-0.030988048762083054
loss:-0.03166618570685387


In [267]:
all_actions_one_hot, all_predicted_actions = generate_event(generator, length=20)
all_actions_one_hot = all_actions_one_hot[:, 1:,]
all_actions_one_hot
# policy gradient loss
negative_likelihoods = tf.keras.losses.categorical_crossentropy\
    (all_actions_one_hot, all_predicted_actions,from_logits=True)
in_dis = tf.argmax(all_actions_one_hot, axis = -1)
print(in_dis.shape)
discriminator(in_dis)
q_values = tf.broadcast_to(discriminator(in_dis), negative_likelihoods.shape) # this needs N-time Monte Carlo search
weighted_negative_likelihoods = tf.multiply(negative_likelihoods, q_values)
loss = tf.reduce_mean(weighted_negative_likelihoods)
    
print(negative_likelihoods.shape)
print(in_dis.shape)
print(q_values.shape)
q_values

(50, 20)
(50, 20)
(50, 20)
(50, 20)


<tf.Tensor: shape=(50, 20), dtype=float32, numpy=
array([[-0.03283375, -0.03283375, -0.03283375, -0.03283375, -0.03283375,
        -0.03283375, -0.03283375, -0.03283375, -0.03283375, -0.03283375,
        -0.03283375, -0.03283375, -0.03283375, -0.03283375, -0.03283375,
        -0.03283375, -0.03283375, -0.03283375, -0.03283375, -0.03283375],
       [-0.02649174, -0.02649174, -0.02649174, -0.02649174, -0.02649174,
        -0.02649174, -0.02649174, -0.02649174, -0.02649174, -0.02649174,
        -0.02649174, -0.02649174, -0.02649174, -0.02649174, -0.02649174,
        -0.02649174, -0.02649174, -0.02649174, -0.02649174, -0.02649174],
       [-0.03558667, -0.03558667, -0.03558667, -0.03558667, -0.03558667,
        -0.03558667, -0.03558667, -0.03558667, -0.03558667, -0.03558667,
        -0.03558667, -0.03558667, -0.03558667, -0.03558667, -0.03558667,
        -0.03558667, -0.03558667, -0.03558667, -0.03558667, -0.03558667],
       [-0.03349002, -0.03349002, -0.03349002, -0.03349002, -0.03349002

In [ ]:
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = mnist_model(images, training=True)
    
        # Add asserts to check the shape of the output.
        tf.debugging.assert_equal(logits.shape, (32, 10))

        loss_value = loss_object(labels, logits)

    loss_history.append(loss_value.numpy().mean())
    grads = tape.gradient(loss_value, mnist_model.trainable_variables)
    print('grads:{}'.format(grads))
    optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))
    return grads


In [112]:
# for i in range(20):
seq_str, seq_int = generate_event(model)
#     print(seq_str)


start view view view view click view view start view install install view click view start install start install click view 
start start click view install start view click view view view install click start view view install install view view na 
start click install install click install na click na na na na na na na na na na na na na 
start click click view view view view install start na install click view view start start start install start click view 
start view start view start click click click install click view install start view install start click start start view install 
start start start click view view view na na na na na na na na na na na na na na 
start view view start start view view view install click start start start click view install click view click click view 
start view view click install start view view click start install click view install start click click view start view install 
start view install view click view install start view click install click v

In [113]:
seq_int

[2, 1, 3, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 1, 2, 2, 2, 3, 4, 4]

In [127]:
# Discriminator - CNN model
cnn_model = Sequential()
cnn_model.add(Embedding(len(vocab), 16, input_length=20))
cnn_model.add(Conv1D(32, 3, activation='relu'))
cnn_model.add(MaxPooling1D(3))
cnn_model.add(Conv1D(32, 3, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(1))
cnn_model.summary()
cnn_model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy',metrics=['acc'])
# history = model.fit(x_train, y_train,
# epochs=10, batch_size=128, validation_split=0.2)
cnn_model.predict(np.array([seq_int]))

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 20, 16)            80        
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 18, 32)            1568      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 6, 32)             0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 4, 32)             3104      
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 33        
Total params: 4,785
Trainable params: 4,785
Non-trainable params: 0
___________________________________________________

array([[0.02621368]], dtype=float32)

In [97]:
class Environment(object):
    def __init__(self, discriminator, rollout_policy, path):
        self.path = path
        self.discriminator = discriminator
        self.mdp = rollout_policy # an pretrained RNN model
        self.base_state = [1]
        self.state = self.base_state
        self.reset()
        self.T = 20
        self.n_sample = 100
    
    def load_model():
        tf.train.latest_checkpoint(checkpoint_dir)
        model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
        model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
        model.build(tf.TensorShape([1, None]))
        model.summary()
        
    def get_state(self):
        return self.state
    
    def reset(self):
        self.t = t
        self.mdp.reset_states()
        self.state = self.base_state
        return True
    
    def step(self, action):
        self.t = self.t + 1

        reward = self.get_reward(action, self.n_sample)
        is_episode_end = self.t > self.T

        self.state.append(action)
        next_state = self.get_state()

        return [next_state, reward, is_episode_end]
    
    def render(self):
        for c in self.state:
            
        full_seq_str.append(idx2str[predicted_idx] + " ")
        full_seq_int.append(predicted_idx)

    return 'start ' + ''.join(full_seq_str), full_seq_int


    def get_reward(self, action, n_sample):
        reward = 0
        for idx_sample in range(n_sample):
            full_seq = self.generate_event(state, self.T)
            reward += self.discriminator.predict(full_seq) / n_sample
        return reward
        
    def generate_event(state, length):
        input_eval = state
        input_eval = tf.expand_dims(input_eval, 0)
        full_seq = input_eval
        # Empty string to store our results
        text_generated = []

        self.rollout_policy.reset_states()
        for i in range(length):
            predictions = mdp(input_eval)
            predictions = tf.squeeze(predictions, 0)
            predictions = predictions
            predicted_idx = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            
            # pass the predicted action as the next input to the model
            input_eval = tf.expand_dims([predicted_idx], 0)
            text_generated.append(idx2str[predicted_idx])
            full_seq.append(predicted_idx)
        return full_seq

    